In [13]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [14]:
# https://www.hanbit.co.kr/academy/books/new_book_list.html?page=1&cate_cd=&srt=&searchKey=&keyWord=
# https://www.hanbit.co.kr/academy/books/book_view.html?p_code=B7192437175
# location='/academy/books/book_view.html?p_code=B7192437175'

In [15]:
base_url = 'https://hanbit.co.kr'
main_url = '/academy/books/new_book_list.html'
url = f'{base_url}{main_url}?page=1'
result = requests.get(url)
soup = BeautifulSoup(result.text)

In [16]:
lis = soup.select('li.sub_book_list')
len(lis)

20

In [17]:
li = lis[0]
href = li.select_one('.pop_quick_bg')['onclick']
href

"location='/academy/books/book_view.html?p_code=B7192437175'"

In [18]:
href[10:-1]

'/academy/books/book_view.html?p_code=B7192437175'

In [19]:
sub_url = f'{base_url}/{href}'
res = requests.get(sub_url)
book_soup = BeautifulSoup(res.text)

In [20]:
res.text

'<!DOCTYPE html>\n<html lang="ko">\n<head>\n  <meta charset="UTF-8">\n  <meta name="viewport" content="width=device-width, initial-scale=1.0">\n  <title>404 Not Found - 한빛출판네트워크</title>\n  <style>\n    html {\n      font-family: Helvetica Neue, Apple SD Gothic Neo, arial, \'나눔고딕\', Nanum Gothic, \'돋움\', Dotum, Tahoma, Geneva, sans-serif;\n    }\n    body {\n      margin: 0;\n    }\n    ol, ul {\n      list-style: none;\n      padding: 0;\n    }\n    h1 {\n      font-size: inherit;\n    }\n    a {\n      text-decoration: none;\n    }\n    .PageError {\n      text-align: center;\n      padding: 60px 0;\n    }\n    .Error_Title {\n      margin: 8px 0;\n      font-size: 60px;\n      color: #303538;\n    }\n    .Error_Description {\n      margin: 20px 0;\n      font-size: 18px;\n      line-height: 1.6em;\n      color: #525A61;\n    }\n    .Error_ButtonWrapper {\n      margin-top: 36px;\n    }\n    .Error_ButtonGroup .Error_ButtonWrapper {\n      display: inline-block;\n    }\n    .Error_But

In [21]:
from tqdm import tqdm
lines = []
for li in tqdm(lis):
    href = li.find('a')['href']
    sub_req = requests.get(base_url + href)
    sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

    title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
    info_lis = sub_soup.select('.info_list > li')
    info_dict = {}
    for li in info_lis:
        key = li.find('strong').get_text().split(' ')[0]
        val = li.find('span').get_text().strip()
        info_dict[key] = val

    author = info_dict['저자']
    try:
        translator = info_dict['번역']
    except:
        translator = '-'
    date = info_dict['출간']
    page = int(info_dict['페이지'].split(' ')[0].replace(',','')) 
    price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',',''))
    lines.append([title, author, translator, date, page, price])  

100%|██████████| 20/20 [00:12<00:00,  1.56it/s]


In [22]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','쪽수','판매가'])
df.tail(20)

,제목,저자,번역,출간일,쪽수,판매가
0,처음 배우는 매트랩,방성완,-,2023-04-10,416,30000
1,SAS와 메타분석을 활용한 한번에 적용하는 의학보건통계,히든그레이스 데이터분석팀,-,2023-03-09,516,34000
2,정보교과교육론(3판),"최현종 , 전용주",-,2023-03-01,488,30000
3,"IT CookBook, 기초전자실험 with PSpice(3판)",홍순관,-,2023-02-28,308,22000
4,경제수학 강의(3판),김성현,-,2023-02-28,464,23000
5,글로벌 사회와 리더십(2판),안광현,-,2023-02-28,416,28000
6,행동 재무론,고광수,-,2023-02-28,596,32000
7,"IT CookBook, 알기 쉽게 해설한 파이썬 with 컴퓨팅 사고력","김충석 , 김지연",-,2023-02-24,328,24000
8,MATLAB으로 배우는 공학 수치해석(3판),방성완,-,2023-02-20,360,28000
9,Cengel의 알기 쉬운 열역학(6판),"Yunus A. Çengel , John M. Cimbala , Afshin J. ...","김택영 , 한진호 , 김덕종 , 김우태 , 이무연 , 전문수",2023-02-17,528,34000


In [23]:
from tqdm import tqdm
lines = []
for page in tqdm(range(1,24)):
    url = f'{base_url}{main_url}?page={page}'
    result = requests.get(url)
    soup = BeautifulSoup(result.text)
    lis = soup.select('li.sub_book_list')
    
    for li in lis:
        href = li.find('a')['href']
        sub_req = requests.get(base_url + href)
        sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

        title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
        info_lis = sub_soup.select('.info_list > li')
        info_dict = {}
        for li in info_lis:
            key = li.find('strong').get_text().split(' ')[0]
            val = li.find('span').get_text().strip()
            info_dict[key] = val

        author = info_dict['저자']
        try:
            translator = info_dict['번역']
        except:
            translator = '-'
        date = info_dict['출간']
        page = int(info_dict['페이지'].split(' ')[0].replace(',','')) 
        price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',',''))
        lines.append([title, author, translator, date, page, price])  

100%|██████████| 23/23 [04:46<00:00, 12.47s/it]


In [26]:
df2 = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','쪽수','판매가'])
df2.tail(10)

,제목,저자,번역,출간일,쪽수,판매가
443,"IT CookBook, 반도체 공학 연습문제 풀이집",김동명,-,2013-05-31,336,22000
444,"IT CookBook, C 프로그래밍","강동진, 하숙정, 임수연, 송무희, 박보석",-,2013-03-20,652,24000
445,"IT CookBook, 컴퓨터 활용 : 윈도우 7, 한글 2010, 엑셀 2010,...","김주영 , 정영준",-,2013-03-20,496,21000
446,"IT CookBook, 초보자의 첫 번째 엑셀 2010",양숙희,-,2013-03-12,152,10000
447,"IT CookBook, 개념이 보이는 물리전자공학",이현용,-,2013-02-28,416,24000
448,문제해결력을 키우는 이공계 글쓰기 : 발표와 토론까지,한성우,-,2013-02-28,396,18000
449,Essential Calculus,James Stewart,-,2012-07-08,960,36000
450,의사결정회계,정재권,-,2011-08-25,506,28000
451,"IT CookBook, 디지털 통신 시스템 : Digital Communicatio...","정호영 , 임승각",-,2011-01-31,576,25000
452,인간관계론,"유기현 , 송병선 , 권용만",-,2008-09-15,415,25000


In [27]:
df2.to_csv('data/20230425_한빛.csv',index=False)